In [1]:
from util.FusionDataGenerator import FusionDataGenerator
import numpy as np
import cmath

import os
import multiprocessing
import argparse
import itertools

import numpy as np
from numpy.random import SeedSequence, default_rng
import torch
from torch import Tensor
import matplotlib.pyplot as plt
from util.load_config_files import load_yaml_into_dotdict
from util.misc import NestedTensor

In [ ]:
datagen = FusionDataGenerator('/home/weixinwei/task.yaml')

In [ ]:
result = datagen.GenMeasurements()

In [ ]:
taskPath = '/home/weixinwei/task.yaml'
modelPath = '/home/weixinwei/study/MT3-test/configs/models/mt3.yaml'
mdc = MT3DataConvertor(taskPath, modelPath)

In [ ]:
mdc.params.totalArg.batchSize

In [2]:
class MT3DataConvertor():
	def __init__(self, taskPath, modelPath) -> None:
		# 从yaml载入超参数
		self.params = load_yaml_into_dotdict(taskPath)
		self.params.update(load_yaml_into_dotdict(modelPath))
		self.__batchSize = int(self.params.totalArg.batchSize)
		self.__nTimeStep = int(self.params.totalArg.n_timesteps)
		self.cycle = int(np.floor((self.params.targetArg.tf - self.params.targetArg.t0) * 1000 / self.params.sensorArg.Radar.T) - self.__nTimeStep + 1)
		self.device = self.params.training.device
		self.fusionDataGenerator = FusionDataGenerator(taskPath)
		self.ResetBias()

	def ResetBias(self, bias=0):
		self.__get_Seq_training_example = self.__Get_Seq_training_example()
		for i in range(bias):
			next(self.__get_Seq_training_example)
		print('DataGeneratorParams: TimeStep = %d, Cycle = %d, Bias = %d.' %(self.__nTimeStep, self.cycle, bias))
		
	def Get_batch(self):
		training_data, labels, unique_ids = next(self.__get_Seq_training_example)

		labels = [Tensor(l).to(torch.device(self.device)) for l in labels]
		unique_ids = [list(u) for u in unique_ids]

		# Pad training data
		max_len = max(list(map(len, training_data)))
		training_data, mask = self.__pad_to_batch_max(training_data, max_len)

		# Pad unique ids
		for i in range(len(unique_ids)):
			unique_id = unique_ids[i]
			n_items_to_add = max_len - len(unique_id)
			unique_ids[i] = np.concatenate([unique_id, [-2] * n_items_to_add])[None, :]
		unique_ids = np.concatenate(unique_ids)

		training_nested_tensor = NestedTensor(Tensor(training_data).to(torch.device(self.device)),
											  Tensor(mask).bool().to(torch.device(self.device)))
		unique_ids = Tensor(unique_ids).to(self.device)

		for i in range(self.__batchSize):
			if torch.all(training_nested_tensor.tensors[i, :, 2] == 0):
				self.ResetBias()
				return self.Get_batch()
			
		return training_nested_tensor, labels, unique_ids
	
	def __Get_Seq_training_example(self):
		while True:
			self.fusionData = self.fusionDataGenerator.GenMeasurements()
			for self.bias in range(self.cycle):
				yield self.__Get_batch_training_example(self.fusionData, self.bias)

	def __Get_batch_training_example(self, result, bias):
		batchSize = self.__batchSize
		nTimestep = self.__nTimeStep
		training_data = []
		labels = []
		unique_ids = []

		for k in range(batchSize):
			tk = result[k]['Radar']['t'][bias : bias+nTimestep]
			Zk = result[k]['Radar']['Z'][bias : bias+nTimestep]
			Z_Truth_k = result[k]['Radar']['Z_Truth'][bias+nTimestep]
			training_data_k = np.empty([0, 3])
			unique_ids_k = np.empty(0).astype(np.int64)
			for n in range(nTimestep):
				Z = Zk[n]
				ZX, ZY = self.__Polar2Rect(Z[1], Z[2])
				ZTX, ZTY = self.__Polar2Rect(Z_Truth_k[1], Z_Truth_k[2])
				training_data_n = np.c_[ZX[:, np.newaxis], ZY[:, np.newaxis], np.zeros([ZX.shape[0], 1]) + tk[n] - tk[0]]
				training_data_k = np.r_[training_data_k, training_data_n]
				unique_ids_k  = np.append(unique_ids_k, Z[0].astype(np.int64))
			# end for
			training_data.append(training_data_k)
			unique_ids.append(unique_ids_k)
			labels.append(np.c_[ZTX[:, np.newaxis], ZTY[:, np.newaxis]])
		return training_data, labels, unique_ids
	
	def __pad_to_batch_max(self, training_data, max_len):
		batch_size = len(training_data)
		d_meas = training_data[0].shape[1]
		training_data_padded = np.zeros((batch_size, max_len, d_meas))
		mask = np.ones((batch_size, max_len))
		for i, ex in enumerate(training_data):
			training_data_padded[i,:len(ex),:] = ex
			mask[i,:len(ex)] = 0

		return training_data_padded, mask
	
	def __Polar2Rect(self, radii, angles):
		rect = radii * np.exp(1j * angles)
		return np.real(rect), np.imag(rect)
	
taskPath = '/home/weixinwei/task.yaml'
modelPath = '/home/weixinwei/study/MT3-test/configs/models/mt3.yaml'
mdc = MT3DataConvertor(taskPath, modelPath)
# training_nested_tensor, labels, unique_ids = mdc.Get_batch()

DataGeneratorParams: TimeStep = 20, Cycle = 31, Bias = 0.


In [19]:
result = mdc.fusionData
batchSize = 16
nTimestep = 20
Ttraining_data = []
Tlabels = []
Tunique_ids = []
bias = 27

for k in range(batchSize):
	tk = result[k]['Radar']['t'][bias : bias+nTimestep]
	Zk = result[k]['Radar']['Z'][bias : bias+nTimestep]
	Z_Truth_k = result[k]['Radar']['Z_Truth'][bias+nTimestep]
	training_data_k = np.empty([0, 3])
	unique_ids_k = np.empty(0).astype(np.int64)
	for n in range(nTimestep):
		Z = Zk[n]
		ZX, ZY = Polar2Rect(Z[1], Z[2])
		ZTX, ZTY = Polar2Rect(Z_Truth_k[1], Z_Truth_k[2])
		training_data_n = np.c_[ZX[:, np.newaxis], ZY[:, np.newaxis], np.zeros([ZX.shape[0], 1]) + tk[n] - tk[0]]
		training_data_k = np.r_[training_data_k, training_data_n]
		unique_ids_k  = np.append(unique_ids_k, Z[0].astype(np.int64))
	# end for
	Ttraining_data.append(training_data_k)
	Tunique_ids.append(unique_ids_k)
	Tlabels.append(np.c_[ZTX[:, np.newaxis], ZTY[:, np.newaxis]])

In [276]:
np.around(np.random.normal(1, 0)).astype(np.int16)

1

In [228]:
# Ttraining_data[1]
Tunique_ids[0].shape

(20,)

In [17]:
def Polar2Rect(radii, angles):
	rect = radii * np.exp(1j * angles)
	return np.real(rect), np.imag(rect)

In [3]:
for i in range(1000):
	training_nested_tensor, labels, unique_ids = mdc.Get_batch()
	if  torch.any(training_nested_tensor.tensors[:, 1:, 2] == 0):
		print(i)
		break

273


In [13]:
training_nested_tensor, labels, unique_ids = mdc.Get_batch()

In [7]:
f = 0
for n in range(10000):
	training_nested_tensor, labels, unique_ids = mdc.Get_batch()
	for i in range(16):
		if torch.all(training_nested_tensor.tensors[i, :, 2] == 0):
			f = 1
	if f == 1:
		print(i)
		break
# # training_nested_tensor.mask[1]
# mdc.fusionData[1]['Radar']['Z']
# mdc.fusionData
# mdc.bias

In [ ]:
Z = result[0]['Radar']['Z'][50]
Z_Truth = result[0]['Radar']['Z_Truth'][50]

In [ ]:
def Polar2Rect(radii, angles):
    return radii * np.exp(1j*angles)

In [ ]:
print(Z[1])
print(Z[2])

In [ ]:
np.append([a1, a2])

In [ ]:
# cmath.polar(complex(3, 4))
cmath.rect(Z[1].tolist(), Z[2].tolist())